# Target Accrual Redemption Forward (TARF) Pivot

This notebook demonstrates how to access and use the functionalities of **Target Accrual Redemption Forward (TARF)** which are part of our **QPS** module within LSEG Financial Analytics SDK.

**You will be able to:**
* Define TARF instrument with basic parameters
* Configure pricing parameters
* Evaluate TARF Analytics

TARF Pivot - A schedule of fixing dates is defined by its starting fixing date, expiry and frequency. The Notional Amount can be entered only in FOR and represents the total notional for the ITM side of the deal. The total notional amount is divided equally on all fixing dates (unweighted) or according to user-defined weights (weighted).

**Three strikes are provided, a central pivot and lower and upper strikes. The inner area in between the lower and upper strikes is considered ITM, while the outer area is considered OTM.**

For every fixing date where the fixing is ITM, the holder enters into a forward @ the relevant outer strike, for an amount equal to the notional amount allocated to said fixing date (expressed in FOR). In particular:

- lower strike < fixing < pivot the holder buys @ lower strike
- pivot < fixing < upper strike the holder sells @ higher strike

Therefore, the holder earns the difference Fixing Notional x (Fixing- Outer Strike) in DOM.

For every fixing date where the fixing is OTM, the holder enters into a forward @ relevant outer strike for a multiple (Leverage) of the allocated notional amount on said fixing date. In particular:

- fixing < lower strike the holder buys @ lower strike
- fixing > upper strike the holder sells @ higher strike

Therefore, the holder loses the difference fixing notional x (Fixing- Outer Strike) x Leverage in DOM.

The positive side of the deal on fixing dates where the fixing is ITM are progressively cumulated. At any time the cumulated positive payoff reaches or surpasses a pre-defined target, the instrument is terminated. Hence, it is possible that the instrument may be subject to an early redemption. The target is defined in terms of conventional 'Big Figures'. The scale of of 1BF is defined on a per-ccypair basis.

An enumerator (Payment Mode) determines how to treat the cumulated payoff amount when exceeding the target. If the mode is set to 'Full Payment', the full amount is paid even for the part exceeding the target. If the mode is set to 'Capped Payment', the final payment exceeding the target is capped, so that the cumulative total earned is exactly the target.

Notice that the target conventionally does not take into account notional weights. It is defined purely as the cumulated 'market movement' on the side which is beneficial to the holder. For a simple unweighted instrument, the target also represents the amount earned by the holder. However, for a weighted instrument, the target does not represent exactly the 'profit' of the holder.

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp

import datetime as dt
import pandas as pd
import json
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example pivot TARF, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

TARF_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "TARF_Pivot",
    inputs = [
        sp.NameTypeValue(name="Underlying", type = "string", value="USDEUR"),
        sp.NameTypeValue(name="StartDate", type = "date", value= dt.date(2022, 1, 25)),
        sp.NameTypeValue(name="EndDate", type = "string", value="StartDate + 1Y"),
        sp.NameTypeValue(name="Frequency", type = "string", value="1M"),
        sp.NameTypeValue(name="Notional", type = "string", value="1000000"),
        sp.NameTypeValue(name="UpperStrike", type = "string", value="0.9"),
        sp.NameTypeValue(name="Pivot", type = "string", value="0.85"),
        sp.NameTypeValue(name="LowerStrike", type = "string", value="0.8"),
        sp.NameTypeValue(name="Leverage", type = "string", value="2"),
        sp.NameTypeValue(name="ProfitTarget", type = "string", value="0.05"),
        sp.NameTypeValue(name="KO_Payment", type = "string", value="Settlement[t]"),
    ],
    payoff_description = [
					[
						"ScheduleType",
						"Schedule description",
						"FX",
						"BuyerSideCoupon",
						"SellerSideCoupon",
						"Settlement",
						"Sum",
						"Alive",
						"KO_Amount",
						"Price"
					],
					[
						"AtDate",
						"StartDate",
						"",
						"",
						"",
						"",
						"0",
						"1",
						"",
						""
					],
					[
						"OnSchedule",
						"DateTable(StartDate + Frequency, EndDate, Frequency, ResetGap := 0b)",
						"FxSpot(Underlying)",
						"If(FX[t] < Pivot, If(FX[t] < LowerStrike, Leverage  * Notional * (FX[t] - LowerStrike), Notional * (FX[t] - LowerStrike)), 0)",
						"If(FX[t] >= Pivot, If(FX[t] < UpperStrike, Notional * (UpperStrike - FX[t]), Leverage * Notional * (UpperStrike - FX[t])), 0)",
						"BuyerSideCoupon[t] + SellerSideCoupon[t]",
						"Sum[LastDate-1] + If(FX[t] < Pivot, Max(FX[t] - LowerStrike, 0), Max(UpperStrike - FX[t], 0))",
						"If(Sum[t] >= ProfitTarget, 0, Alive[LastDate - 1])",
						"Alive[LastDate - 1] * (1 - Alive[LastDate]) * KO_Payment",
						"Alive[t] * Settlement[t] + KO_Amount[t]"
					],
					[
						"AtDate",
						"EndDate",
						"FxSpot(Underlying)",
						"If(FX[t] < Pivot, If(FX[t] < LowerStrike, Leverage  * Notional * (FX[t] - LowerStrike), Notional * (FX[t] - LowerStrike)), 0)",
						"If(FX[t] >= Pivot, If(FX[t] < UpperStrike, Notional * (UpperStrike - FX[t]), Leverage * Notional * (UpperStrike - FX[t])), 0)",
						"BuyerSideCoupon[t] + SellerSideCoupon[t]",
						"Sum[LastDate-1] + If(FX[t] < Pivot, Max(FX[t] - LowerStrike, 0), Max(UpperStrike - FX[t], 0))",
						"If(Sum[t] >= ProfitTarget, 0, Alive[LastDate - 1])",
						"Alive[LastDate - 1] * (1 - Alive[LastDate]) * KO_Payment",
						"Alive[t] * Settlement[t] + KO_Amount[t]"
					]
				]
)


print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")

tarf_pivot = sp.StructuredProductsDefinitionInstrument(definition = TARF_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

TARF_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2022, 3, 16),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "USDEUR",
            underlying_tag = "USDEUR",
            underlying_currency = "EUR",
            asset_class = "ForeignExchange",
            model_name= "Heston")]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price function
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = sp.price(
        definitions=[tarf_pivot],
        pricing_preferences=TARF_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing Execution Successful!")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing Execution Successful!


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation, cash-flows and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "TARF_Pivot",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 8.496792413,
        "BuyerSideCoupon": -15292.387728856,
        "FX": 9.916931107,
        "KO_Amount": 4398.522262662,
        "Price": -353945.676181702,
        "SellerSideCoupon": -295357.648603112,
        "Settlement": -310650.036331968,
        "Sum": 0.365314059
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_tarf_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_tarf_valuation)

,Field,Value
0,marketValueInDealCcy,-353945.676182


### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Build dataframes for all cash flow types
df_fx = pd.DataFrame(cashflows[0]['payments']).rename(columns={'amount': cashflows[0]['legTag']})
df_BuyerSideCoupon = pd.DataFrame(cashflows[1]['payments']).rename(columns={'amount': cashflows[1]['legTag']})
df_SellerSideCoupon = pd.DataFrame(cashflows[2]['payments']).rename(columns={'amount': cashflows[2]['legTag']})
df_settlement = pd.DataFrame(cashflows[3]['payments']).rename(columns={'amount': cashflows[3]['legTag']})
df_sum  = pd.DataFrame(cashflows[4]['payments']).rename(columns={'amount': cashflows[4]['legTag']})
df_alive = pd.DataFrame(cashflows[5]['payments']).rename(columns={'amount': cashflows[5]['legTag']})
df_ko_amount = pd.DataFrame(cashflows[6]['payments']).rename(columns={'amount': cashflows[6]['legTag']})
df_price = pd.DataFrame(cashflows[7]['payments']).rename(columns={'amount': cashflows[7]['legTag']})

# Merge all dataframes on the 'date' column
# Add suffix to distinguish between different data sources
combined_df = pd.concat(
    [df_fx.add_suffix('_fx'), 
     df_BuyerSideCoupon.add_suffix('_buyer_coupon'),
     df_SellerSideCoupon.add_suffix('_seller_coupon'),
     df_settlement.add_suffix('_settlement'), 
     df_sum.add_suffix('_sum'), 
     df_alive.add_suffix('_alive'), 
     df_ko_amount.add_suffix('_ko_amount'), 
     df_price.add_suffix('_price')], 
    axis=1
)

# Remove duplicate columns with the same values
combined_df = combined_df.loc[:, ~combined_df.T.duplicated()]

# Keep only one discountFactor column and sort the columns
combined_df = combined_df.loc[:,['date_fx', 'discountFactor_fx', 'FX_fx', 'BuyerSideCoupon_buyer_coupon', 'SellerSideCoupon_seller_coupon', 'Settlement_settlement','Sum_sum',  'Alive_alive',
       'KO_Amount_ko_amount','Price_price', 'currency_fx', 'event_fx','occurence_fx']]

# Rename columns to match the desired layout
combined_df.columns = ['date', 'discountFactor', 'FX', 'BuyerSideCoupon', 'SellerSideCoupon', 'Settlement', 'Sum', 'Alive', 'KO_Amount', 'Price', 'currency', 'event', 'occurrence']

# Display the combined dataframe
display(combined_df)

,date,discountFactor,FX,BuyerSideCoupon,SellerSideCoupon,Settlement,Sum,Alive,KO_Amount,Price,currency,event,occurrence
0,2022-01-25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,EUR,Payoff,Historical
1,2022-02-25,1.000000,0.887390,0.000000,12609.814535,12609.814535,0.012610,1.0000,0.000000,12609.814535,EUR,Payoff,Historical
2,2022-03-25,1.000123,0.906117,0.000000,-15260.371543,-15260.371543,0.003025,1.0000,0.000000,-15260.371543,EUR,Payoff,Future
3,2022-04-25,1.000636,0.905213,830.041180,-20879.767839,-20049.726658,0.019724,0.9530,2031.272031,-20049.726658,EUR,Payoff,Future
4,2022-05-25,1.001116,0.904112,1141.022827,-24172.713114,-23031.690287,0.011217,0.9632,815.169007,-23677.156631,EUR,Payoff,Future
5,2022-06-27,1.001633,0.902777,401.550098,-26649.483287,-26247.933189,0.028644,0.8266,3595.400193,-27373.453699,EUR,Payoff,Future
6,2022-07-25,1.002044,0.901621,-308.888435,-28010.816782,-28319.705217,0.020581,0.8402,709.674917,-32391.168771,EUR,Payoff,Future
7,2022-08-25,1.002452,0.900025,-978.774455,-28539.421364,-29518.195819,0.038556,0.7158,1052.453130,-33978.037950,EUR,Payoff,Future
8,2022-09-26,1.002831,0.898307,-1962.582050,-29380.195024,-31342.777074,0.031259,0.7392,20.982540,-37505.678455,EUR,Payoff,Future
9,2022-10-25,1.003093,0.896887,-2808.014828,-30125.019503,-32933.034331,0.049806,0.6310,-779.025515,-39037.733124,EUR,Payoff,Future


### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_tarf_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_tarf_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,-122721.00
1,gammaAmountInDealCcy,-23438.70
2,vegaAmountInDealCcy,-68835.00
3,thetaAmountInDealCcy,1462.84
